## Cài đặt và import các thư viện cần thiết

In [1]:
! pip install -qq unsloth==2025.4.1 vllm==0.7.3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.4/46.4 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.2/193.2 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.5/96.5 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.6/87.6 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.7/16.7 MB 81.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.9/396.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211

In [2]:
import torch
from trl import SFTTrainer
from datasets import load_dataset
from unsloth import FastLanguageModel
from transformers import GenerationConfig
from transformers import TrainingArguments

from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_value_0 = user_secrets.get_secret("push_to_hub")

2025-05-09 03:56:47.104195: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746763007.303184      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746763007.359598      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
/tmp/ipykernel_19/3302373450.py:4: UserWarning: WARNING: Unsloth should be imported before trl, transformers, peft to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


Unsloth: Patching Xformers to fix some performance issues.
🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 05-09 03:57:06 __init__.py:207] Automatically detected platform cuda.


In [3]:
from huggingface_hub import login
login(token=secret_value_0)

## Khởi tạo model

In [4]:
MAX_SEQ_LENGTH = 2048
LORA_RANK = 16
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Llama-3.2-1B-Instruct",
    max_seq_length=MAX_SEQ_LENGTH,
    load_in_4bit=True,
    fast_inference=True,
    max_lora_rank=LORA_RANK,
    dtype=torch.float16
)

model = FastLanguageModel.get_peft_model(
    model,
    r=LORA_RANK,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=[
        "q_proj", "k_proj", "v_proj", "up_proj",
        "down_proj", "o_proj", "gate_proj"],
    use_rslora=True,
    use_gradient_checkpointing="unsloth",
    random_state=42,
)


==((====))==  Unsloth 2025.4.1: Fast Llama patching. Transformers: 4.51.1. vLLM: 0.7.3.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/llama-3.2-1b-instruct-unsloth-bnb-4bit with actual GPU utilization = 49.53%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 192.
Unsloth: vLLM's KV Cache can use up to 6.22 GB. Also swap space = 5 GB.
WARNING 05-09 03:57:16 config.py:2448] Casting torch.bfloat16 to torch.float16.
INFO 05-09 03:57:28 config.py:549] This model supports multiple tasks: {'generate', 'classify', 'reward', 'sco

tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/234 [00:00<?, ?B/s]

...

INFO 05-09 03:58:41 model_runner.py:1562] Graph capturing finished in 33 secs, took 0.34 GiB
INFO 05-09 03:58:41 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 49.02 seconds


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

Unsloth 2025.4.1 patched 16 layers with 16 QKV layers, 16 O layers and 16 MLP layers.


## Tải và tiền xử lý multiturn conversation datasets

In [5]:
dataset_name = "5CD-AI/Vietnamese-Multi-turn-Chat-Alpaca"
raw_dataset = load_dataset(dataset_name, split="train")
raw_dataset[0]

README.md:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

vi_chatalpaca_cleaned.json:   0%|          | 0.00/69.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/12697 [00:00<?, ? examples/s]

{'conversations': [{'from': 'human',
   'value': 'Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."'},
  {'from': 'gpt',
   'value': 'Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.'},
  {'from': 'human',
   'value': 'Bạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?'},
  {'from': 'gpt',
   'value': 'Chắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường độ và sự tập trung của các buổi tập để đảm bảo mọi cầu thủ đều được chuẩn bị đầy đủ về thể chất và tinh thần. 3. Luyện tập những kỹ năng cụ thể cần cải thiện, chẳng hạn như sút bóng hoặc chuyền bóng chính xác. 4. Phát triển các chiến lược và chiến thuật của đ

In [6]:
SYS_INSTRUCT = "Bạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt."
def convert_to_chat_format(conversations):
    messages = [{"role": "system", "content": SYS_INSTRUCT}]
    for msg in conversations:
        role = "user" if msg["from"] == "human" else "assistant"
        messages.append({"role": role, "content": msg["value"]})

    return messages

def format_prompt(example):
    messages = convert_to_chat_format(example["conversations"])
    return {
        "text": tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=False
        )
    }

def tokenize_function(examples):
    return tokenizer(examples["text"], max_length = MAX_SEQ_LENGTH, padding = "max_length", truncation = True)

raw_dataset = raw_dataset.map(format_prompt, remove_columns=raw_dataset.column_names)
dataset = raw_dataset.map(tokenize_function, batched = True)

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

Map:   0%|          | 0/12697 [00:00<?, ? examples/s]

In [7]:
dataset[0]["text"]

'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 09 May 2025\n\nBạn là một trợ lý AI thân thiện, hãy trả lời bằng tiếng Việt.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nHãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "Trận đấu là một thất bại nặng nề mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần."<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nNhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nBạn có thể đề xuất một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của họ trong trận đấu tiếp theo không?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nChắc chắn, đây là một số chiến lược mà nhóm có thể sử dụng để cải thiện hiệu suất của mình trong trận đấu tiếp theo: 1. Phân tích trận đấu trước bằng cách xem lại cảnh quay trận đấu để xác định điểm yếu và các lĩnh vực cần cải thiện. 2. Tăng cường

In [8]:
import os
os.environ['UNSLOTH_RETURN_LOGITS'] = '1'

training_args = TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=2,
    learning_rate=1e-4,
    save_total_limit=4,
    logging_steps=20,
    output_dir="./checkpoint/llama3-1b-multi-conversation",
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    save_strategy="steps",
    save_steps=50,
    report_to="none",
    remove_unused_columns=True,
    max_steps=500,
    fp16=True,
    # Vì không có tập validation nên kh load_best_model_at_end = True được
)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
)

trainer.train()

/tmp/ipykernel_19/3289367494.py:23: FutureWarning: `tokenizer` is deprecated and removed starting from version 0.16.0 for `SFTTrainer.__init__`. Use `processing_class` instead.
  trainer = SFTTrainer(


Converting train dataset to ChatML (num_proc=4):   0%|          | 0/12697 [00:00<?, ? examples/s]

Applying chat template to train dataset (num_proc=4):   0%|          | 0/12697 [00:00<?, ? examples/s]

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 12,697 | Num Epochs = 1 | Total steps = 500
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 11,272,192/1,000,000,000 (1.13% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
20,1.843600
40,1.550400
60,1.492100
80,1.485200
100,1.439700
120,1.484100
140,1.465100
160,1.414000
180,1.409200
200,1.390600


TrainOutput(global_step=500, training_loss=1.433992733001709, metrics={'train_runtime': 2158.007, 'train_samples_per_second': 0.927, 'train_steps_per_second': 0.232, 'total_flos': 1.2854350577664e+16, 'train_loss': 1.433992733001709})

## Lưu mô hình và inference

In [9]:
model.push_to_hub_merged(
    "TSunm/Llama-3.2-1B-Instruct-Chat-sft",
    commit_message="Merge weights to push to hub",
)

tokenizer.push_to_hub(
    "TSunm/Llama-3.2-1B-Instruct-Chat-sft",
    commit_message="Push tokenizer to hub",
)


Unsloth: You're not saving a tokenizer as well?
You can do it separately via `tokenizer.push_to_hub(...)`
Unsloth: You are pushing to hub in Kaggle environment.
To save memory, we shall move TSunm/Llama-3.2-1B-Instruct-Chat-sft to /tmp/Llama-3.2-1B-Instruct-Chat-sft
Unsloth: You have 2 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 1.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 13.2 out of 31.35 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


100%|██████████| 16/16 [00:00<00:00, 37.99it/s]



Unsloth: Saving /tmp/Llama-3.2-1B-Instruct-Chat-sft/pytorch_model.bin...


README.md:   0%|          | 0.00/61.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/TSunm/Llama-3.2-1B-Instruct-Chat-sft


No files have been modified since last commit. Skipping to prevent empty commit.


In [10]:
generation_config = GenerationConfig(
    max_new_tokens=128,
    temperature=1.0,
    do_sample=False,
    num_return_sequences=1,
    pad_token_id=tokenizer.eos_token_id,
    eos_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.3
)

prompt = [
    {"role": "system", "content": (
        "Bạn là một trợ lý AI thân thiện, "
        "hãy trả lời bằng tiếng Việt."
    )},

    {"role": "user", "content": (
        "Hãy chỉnh sửa câu này để ngắn gọn hơn mà không mất đi ý nghĩa: "
        "\"Trận đấu là một thất bại nặng nề "
        "mặc dù thực tế là cả đội đã tập luyện trong nhiều tuần.\""
    )},

    {"role": "assistant", "content": (
        "Nhiều tuần huấn luyện của đội đã dẫn đến một thất bại nặng nề."
    )},

    {"role": "user", "content": (
        "Bạn có thể đề xuất một số chiến lược mà "
        "nhóm có thể sử dụng để cải thiện hiệu suất "
        "của họ trong trận đấu tiếp theo không?"
    )}
]

chat_text = tokenizer.apply_chat_template(
    prompt,
    add_generation_prompt=True,
    tokenize=False
)

inputs = tokenizer(
    chat_text,
    return_tensors="pt"
).to("cuda:0")

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        generation_config=generation_config,
    )
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

if "assistant\n" in output_text:
    answer = output_text.split("assistant\n")[-1].strip()
else:
    answer = output_text.strip()

print("Assistant reply:", answer)


`generation_config` default values have been modified to match model-specific defaults: {'max_length': 131072, 'bos_token_id': 128000}. If this is not desired, please set these values explicitly.


Assistant reply: Chắc chắn! Dưới đây là một vài mẹo giúp ích cho các nhà tổ chức và cầu thủ khi chuẩn bị cho trận đấu tiếp theo: 1. Tìm hiểu về đối phương - Hãy nghiên cứu thông tin chi tiết về đối thủ như lịch trình, điểm mạnh yếu nhất và những gì chúng đang làm tốt ở sân hoặc trên bàn đá bóng. Điều quan trọng nữa là phải xác định bất kỳ sự khác biệt nào giữa hai đội cũng như cách thức hoạt động của từng người chơi. 2. Đánh giá tình hình sức khỏe tổng thể - Kiểm tra xem tất cả các thành viên đều sẵn sàng tham gia
